In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from math import cos, sin, asin, sqrt, radians
import itertools
%matplotlib inline

In [2]:
# Link to file: https://www.dropbox.com/s/l4ej7s0r3nfs0js/centroids_planning_areas.geojson?dl=0
df = gpd.GeoDataFrame.from_file ('Data/Raw Data/shapefile/44/centroids_planning_areas.geojson')
df.index = df.FID
df.index.name = None

In [7]:
df.columns

Index([u'FID', u'Latitude', u'Longitude', u'Planning_A', u'geometry'], dtype='object')

In [3]:
d = {}
for i in df.FID.values:
    d[i] = df.Planning_A.iloc[i]

In [4]:
times_between_areas=pd.read_csv('https://www.dropbox.com/s/lkpdetlefpsm8bw/times.csv?dl=1')
times_between_areas.drop('Unnamed: 0', axis=1, inplace=True)
times_between_areas.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,0,14,5,21,17,13,14,14,10,22,...,11,16,12,26,6,30,31,21,10,6
1,15,0,14,23,19,13,16,15,25,11,...,20,6,13,26,10,33,31,24,22,18
2,6,13,0,18,15,13,12,11,15,22,...,14,15,10,21,6,29,27,20,13,10
3,22,23,18,0,10,12,13,13,29,28,...,17,24,13,15,18,15,11,10,19,21
4,19,19,14,12,0,13,5,10,26,27,...,11,21,13,13,15,21,20,12,14,17


In [5]:
travel_patterns = pd.read_csv('https://www.dropbox.com/s/l1vm18b90yz09yk/travel_patterns_processd.csv?dl=1')
travel_patterns.rename(columns={'Unnamed: 0': 'Planning_A'}, inplace=True)
travel_patterns.head()

,Planning_A,Up to 15 mins,16 - 30 mins,31 - 45 mins,46 - 60 mins,More than 60 mins,No_transport,TOTAL
0,ANG MO KIO,16.2,32.4,23.6,18.1,7.7,10.3,108.3
1,BEDOK,23.7,56.8,30.4,26.2,9.1,12.6,158.8
2,BISHAN,5.8,19.2,12.7,7.9,2.0,3.6,51.2
3,BUKIT BATOK,9.2,25.7,16.2,16.0,6.7,6.1,79.9
4,BUKIT MERAH,16.8,37.1,12.1,10.4,4.1,8.5,89.0


***Adding those Planning Areas that are not in the travel_pattern data. Substitute their value using "OTHERS".***

In [8]:
key_planning_area = df[['FID','Planning_A']]

In [9]:
areas = list(travel_patterns.Planning_A.values)
tv = list(travel_patterns.as_matrix())
for i in np.unique(df.Planning_A.values):
    if i not in areas:
        new_row = np.insert(np.array(travel_patterns.ix[28].values[1:]),obj=0,values=i)# Contains info for "OTHER" planning areas: travel_patterns['Planning_A'].ix[28]
        tv.append(new_row)
tv = pd.DataFrame(tv,columns=travel_patterns.columns)
travel_with_labels = pd.merge(tv,key_planning_area,on=['Planning_A'],how='inner')
travel_with_labels.columns        

Index([        u'Planning_A',     u'Up to 15 mins ',      u'16 - 30 mins ',
            u'31 - 45 mins ',      u'46 - 60 mins ', u'More than 60 mins ',
             u'No_transport',              u'TOTAL',                u'FID'],
      dtype='object')

In [10]:
def get_pairs(column,a,b):
    list_ = []
    for i,j in itertools.product(times_between_areas.index.values,times_between_areas.columns.values):
        i = int(i)
        j = int(j)
        if i != j:
            if (times_between_areas.iloc[i,j]>a)&(times_between_areas.iloc[i,j]<=b):
                list_.append((i,j,travel_with_labels[travel_with_labels['FID']==i][column].values[0]))
            else:
                pass 
        else:
            pass
    column = column.strip()
    return pd.DataFrame(list_,columns=('FID_origin','FID_destination',column))    

In [11]:
min15 = get_pairs('Up to 15 mins ',0,15)
min30 = get_pairs('16 - 30 mins ',15,30)
min45 = get_pairs('31 - 45 mins ',30,45)
min60 = get_pairs('46 - 60 mins ',45,60)
max60 = get_pairs('More than 60 mins ',60,100000)

In [21]:
min15['FID_origin_Name'] = min15['FID_origin'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
min30['FID_origin_Name'] = min30['FID_origin'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
min45['FID_origin_Name'] = min45['FID_origin'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
min60['FID_origin_Name'] = min60['FID_origin'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
max60['FID_origin_Name'] = max60['FID_origin'].apply(lambda x: str(x).replace(str(x), d[int(x)]))

In [24]:
min15['FID_destination_Name'] = min15['FID_destination'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
min30['FID_destination_Name'] = min30['FID_destination'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
min45['FID_destination_Name'] = min45['FID_destination'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
min60['FID_destination_Name'] = min60['FID_destination'].apply(lambda x: str(x).replace(str(x), d[int(x)]))
max60['FID_destination_Name'] = max60['FID_destination'].apply(lambda x: str(x).replace(str(x), d[int(x)]))

In [30]:
def rename_col(x):
    x.columns = ['FID_origin', 'FID_destination', 'People_in_Transit_per_1000', 'FID_origin_Name',
       'FID_destination_Name']
    return x

In [31]:
data = pd.concat([rename_col(min15),rename_col(min30),rename_col(min45),rename_col(min60),rename_col(max60)],axis=0)

In [32]:
data.head()

,FID_origin,FID_destination,People_in_Transit_per_1000,FID_origin_Name,FID_destination_Name
0,0.0,1.0,16.2,ANG MO KIO,BEDOK
1,0.0,2.0,16.2,ANG MO KIO,BISHAN
2,0.0,5.0,16.2,ANG MO KIO,BUKIT MERAH
3,0.0,6.0,16.2,ANG MO KIO,BUKIT PANJANG
4,0.0,7.0,16.2,ANG MO KIO,BUKIT TIMAH


In [34]:
population_by_age = pd.read_csv('https://www.dropbox.com/s/bhd8j7a8sxkssfc/POP2015.csv?dl=1')

In [36]:
population_by_age.columns

Index([u'Planning_Area', u' Total ', u' 0 - 4 ', u' 5 - 9 ', u' 10 - 14 ',
       u' 15 - 19 ', u' 20 - 24 ', u' 25 - 29 ', u' 30 - 34 ', u' 35 - 39 ',
       u' 40 - 44 ', u' 45 - 49 ', u' 50 - 54 ', u' 55 - 59 ', u' 60 - 64 ',
       u' 65 - 69 ', u' 70 - 74 ', u' 75 - 79 ', u' 80 - 84 ', u' 85 & Over '],
      dtype='object')

In [44]:
# Assuming that all age groups are travelling according to the census data => Use total population
# Prepare for merger
def upper_case(df):
    # Delete observations not containing TOTAL if a TOTAl observation exists
    df['Planning_Area'] = df['Planning_Area'].apply(lambda x: x.replace('-','').upper().replace('TOTAL','').strip())
    df = df.drop_duplicates(subset='Planning_Area')
    df = df.fillna(np.nan)
    return df

population_by_age = upper_case(population_by_age)

In [45]:
# Ignore first row => gives total figures across all of Singapore
population_by_age.head(2)

,Planning_Area,Total,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 & Over
0,,3902690,183580,204450,214390,242900,264130,271030,290620,301070,316760,303410,315090,295060,240490,182430,102630,81210,51790,41660
1,ANG MO KIO,174770,6790,7660,8290,9320,10310,11170,12250,13070,13710,13000,14010,13800,12980,11050,6670,5140,3250,2300


In [48]:
for col in list(population_by_age.columns)[1:]:
    population_by_age[col] = np.where(population_by_age[col].apply(lambda x: x=='  '),np.nan,population_by_age[col])
    population_by_age[col] = population_by_age[col].astype(float)

In [49]:
population = population_by_age[[' Total ','Planning_Area']]
population['Total/1000'] = population[' Total '].astype(float)/1000

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [57]:
data.columns

Index([u'FID_origin', u'FID_destination', u'People_in_Transit_per_1000',
       u'FID_origin_Name', u'FID_destination_Name'],
      dtype='object')

In [63]:
dt = data.merge(population.rename(columns={'Planning_Area':'FID_origin_Name'}),
on='FID_origin_Name',how='left').rename(columns={' Total ':'FID_origin_pop',
'Total/1000':'FID_origin_pop/1000'}).merge(population.rename(columns={'Planning_Area':'FID_destination_Name'}),
how='left',on='FID_destination_Name').rename(columns={' Total ':'FID_destination_pop',
'Total/1000':'FID_destination_pop/1000'})

In [64]:
dt.head()

,FID_origin,FID_destination,People_in_Transit_per_1000,FID_origin_Name,FID_destination_Name,FID_origin_pop,FID_origin_pop/1000,FID_destination_pop,FID_destination_pop/1000
0,0.0,1.0,16.2,ANG MO KIO,BEDOK,174770.0,174.77,289750.0,289.75
1,0.0,2.0,16.2,ANG MO KIO,BISHAN,174770.0,174.77,90700.0,90.70
2,0.0,5.0,16.2,ANG MO KIO,BUKIT MERAH,174770.0,174.77,155840.0,155.84
3,0.0,6.0,16.2,ANG MO KIO,BUKIT PANJANG,174770.0,174.77,139030.0,139.03
4,0.0,7.0,16.2,ANG MO KIO,BUKIT TIMAH,174770.0,174.77,74470.0,74.47


People in transit per 1000 are attributed to the Planning Areas of destination with weights from lot density.
A more naive way of assigning weights would be to just divide the origin population by the number of destinations.

In [71]:
# Each Planning Area is part of a pair 51 times.
pd.DataFrame({'count':dt.groupby('FID_origin_Name').size()}).reset_index()['count'].value_counts()

51    52
Name: count, dtype: int64

In [80]:
dt['Adjusted_origin_pop'] = dt['FID_origin_pop/1000']/51*dt['People_in_Transit_per_1000']
pd.DataFrame({'Working_pop':dt.groupby('FID_destination_Name')['Adjusted_origin_pop'].sum()}).reset_index()

,FID_destination_Name,Working_pop
0,ANG MO KIO,1216.637039
1,BEDOK,1388.224275
2,BISHAN,1224.792902
3,BOON LAY,1971.688863
4,BUKIT BATOK,1799.689647
5,BUKIT MERAH,1248.393255
6,BUKIT PANJANG,1497.790196
7,BUKIT TIMAH,1380.388314
8,CENTRAL WATER CATCHMENT,2356.098549
9,CHANGI,1925.992529
